In [1]:
import pandas as pd
from itertools import combinations

In [ ]:
interactions = pd.read_pickle("../data/processed/kion/test_sequences.pkl")
# inv_mappings_users, inv_mappings_items = pd.read_pickle('../data/processed/kion/mappings.pkl')
# mappings_users = {v: k for k, v in inv_mappings_users.items()}
# mappings_items = {v: k for k, v in inv_mappings_items.items()}

162559

In [ ]:
# Создадим список для хранения количества общих айтемов для каждой пары
user_similarity = []

for (user1, items1), (user2, items2) in combinations(interactions.items(), 2):
    intersection = len(set(items1) & set(items2))  # Найдем пересечение
    user_similarity.append((user1, user2, intersection))

In [ ]:
user_similarity_df = pd.DataFrame(user_similarity, columns=["user1", "user2", "common_items"])

In [ ]:
user_similarity_df.to_csv('./output/user_similarity.csv', index=False)

In [ ]:
most_similar = user_similarity_df.loc[user_similarity_df["common_items"].idxmax()]
most_different = user_similarity_df.loc[user_similarity_df["common_items"].idxmin()]

print("Пара наиболее похожих пользователей:", most_similar)
print("Пара наиболее разных пользователей:", most_different)

Пара наиболее похожих пользователей: user1           4169
user2           4277
common_items    1367
Name: 16486631, dtype: int64
Пара наиболее разных пользователей: user1            1
user2           46
common_items     0
Name: 44, dtype: int64


## Анализ

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np

In [2]:
df = pd.read_json('../embeddings.json')
user_similarity_df = pd.read_csv('./output/user_similarity.csv')

In [3]:
df.head()

,id,embedding
0,1,"[-0.0005691268, 0.021929597000000002, -0.02784..."
1,2,"[-0.0117520755, 0.02627983, -0.000477069959999..."
2,3,"[-0.011479706000000001, 0.037621167000000004, ..."
3,4,"[-0.01733712, 0.044699278, -0.011136063, -0.01..."
4,5,"[-0.0052591999999999995, 0.019822385, -0.01957..."


In [3]:
most_dissimilar_users = user_similarity_df.sort_values(by='common_items', ascending=True).head(50)

In [4]:
min_sim = 1

In [84]:
sim_list = []

In [85]:
for _, row in user_similarity_df.query('common_items == 0').loc[:200000].iterrows():
    dissimilar_user1 = df.query(f'id == {row["user1"]}').embedding.values[0]
    dissimilar_user2 = df.query(f'id == {row["user2"]}').embedding.values[0]
    
    torch_sim = torch.nn.CosineSimilarity(dim=0)(torch.tensor(dissimilar_user1), torch.tensor(dissimilar_user2))
    sklearn_sim = cosine_similarity([dissimilar_user1], [dissimilar_user2])[0][0]
    min_sim = min(min_sim, sklearn_sim)
    sim_list.append(sklearn_sim)
    # print(f'User {row["user1"]} and user {row["user2"]} have {row["common_items"]} common items. Similarity is {sklearn_sim} | {torch_sim}')

In [88]:
np.mean(sim_list)

0.8840421037418708

In [89]:
print(min_sim)

0.7535151120736256


In [36]:
most_similar_users = user_similarity_df.sort_values(by='common_items', ascending=False).head()

In [42]:
for _, row in most_similar_users.iterrows():
    similar_user1 = df.query(f'id == {row["user1"]}').embedding.values[0]
    similar_user2 = df.query(f'id == {row["user2"]}').embedding.values[0]
    print(torch.nn.CosineSimilarity(dim=0)(torch.tensor(similar_user1), torch.tensor(similar_user2)))
    print(f'User {row["user1"]} and user {row["user2"]} have {row["common_items"]} common items. Similarity is {cosine_similarity([dissimilar_user1], [dissimilar_user2])[0][0]}')

tensor(0.9330)
User 4169 and user 4277 have 1367 common items. Similarity is 0.9566605880432233
tensor(0.9376)
User 1680 and user 4169 have 1334 common items. Similarity is 0.9566605880432233
tensor(0.9467)
User 1181 and user 4169 have 1237 common items. Similarity is 0.9566605880432233
tensor(0.9132)
User 1941 and user 4169 have 1191 common items. Similarity is 0.9566605880432233
tensor(0.9567)
User 889 and user 4169 have 1187 common items. Similarity is 0.9566605880432233
